In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh(),
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)


In [ ]:
generator = Generator()
discriminator = Discriminator()

criterion = nn.BCELoss()

optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)


In [ ]:
# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# Define the device for training
device = torch.device('cuda' if cuda_available else 'cpu')


In [ ]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()

if cuda_available:
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

# Optionally, to identify the GPU
if cuda_available:
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# Create the models
generator = Generator().to(device)
discriminator = Discriminator().to(device)


In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        real_images = Variable(images.view(images.size(0), -1)).to(device)
        fake_labels = Variable(torch.zeros(images.size(0))).to(device)
        real_labels = Variable(torch.ones(images.size(0))).to(device)

        # Train the discriminator
        optimizer_d.zero_grad()
        outputs = discriminator(real_images).squeeze()
        real_loss = criterion(outputs, real_labels)
        real_score = outputs

        z = Variable(torch.randn(images.size(0), 100)).to(device)
        fake_images = generator(z)
        outputs = discriminator(fake_images).squeeze()
        fake_loss = criterion(outputs, fake_labels)
        fake_score = outputs

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_d.step()

        # Train the generator
        optimizer_g.zero_grad()
        z = Variable(torch.randn(images.size(0), 100)).to(device)
        fake_images = generator(z)
        outputs = discriminator(fake_images).squeeze()

        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_g.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f'
                  %(epoch, num_epochs, i+1, len(train_loader), d_loss.data, g_loss.data, real_score.data.mean(), fake_score.data.mean()))


In [ ]:
# Save the trained models
torch.save(generator.state_dict(), 'generator_gpu.pth')
torch.save(discriminator.state_dict(), 'discriminator_gpu.pth')


In [ ]:
# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# Define the device for training
device = torch.device('cuda' if cuda_available else 'cpu')

# Create the models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Load the models from disk
generator.load_state_dict(torch.load('generator_gpu.pth', map_location=device))
discriminator.load_state_dict(torch.load('discriminator_gpu.pth', map_location=device))

# Set the models to evaluation mode
generator.eval()
discriminator.eval()


In [ ]:
# Generate fake images
z = Variable(torch.randn(64, 100)).to(device)
fake_images = generator(z)
fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
fake_images = fake_images.data.cpu().numpy()  # move data back to CPU for numpy and plotting
fake_images = fake_images * 0.5 + 0.5  # unnormalize

# Plot the fake images
fig, axes = plt.subplots(8, 8, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    ax.imshow(fake_images[i, 0], cmap='gray')
    ax.axis('off')

plt.show()


In [ ]:
import torch
import torch.onnx
import warnings
from pathlib import Path

# Define the path for the ONNX model
onnx_path = Path('generator.onnx')

# Load the trained generator model
generator = Generator().to(device)
generator.load_state_dict(torch.load('generator_gpu.pth'))
generator.eval()  # Set the model to evaluation mode

# Suppress warnings and export the model to ONNX format if it doesn't already exist
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    if not onnx_path.exists():
        # Assume IMAGE_HEIGHT and IMAGE_WIDTH are both 32 for CIFAR-10 dataset
        IMAGE_HEIGHT, IMAGE_WIDTH = 32, 32
        dummy_input = torch.randn(1, 100, 1, 1).to(device)  # Adjust the input dimensions to match your model
        torch.onnx.export(
            generator,
            dummy_input,
            onnx_path,
        )
        print(f"ONNX model exported to {onnx_path}.")
    else:
        print(f"ONNX model {onnx_path} already exists.")


In [ ]:
import torch
import torch.onnx
import warnings
from pathlib import Path

# Define the path for the ONNX model
onnx_path = Path('generator.onnx')

# Load the trained generator model
generator = Generator().to(device)
generator.load_state_dict(torch.load('generator_gpu.pth'))
generator.eval()  # Set the model to evaluation mode

# Suppress warnings and export the model to ONNX format if it doesn't already exist
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    if not onnx_path.exists():
        # Adjust the input dimensions to match your model
        dummy_input = torch.randn(1, 100).to(device)  # Adjusted input dimensions
        torch.onnx.export(
            generator,
            dummy_input,
            onnx_path,
        )
        print(f"ONNX model exported to {onnx_path}.")
    else:
        print(f"ONNX model {onnx_path} already exists.")


In [ ]:
pip install onnx

In [ ]:
pip install onnxruntime-gpu

In [ ]:
# Install openvino package
%pip install -q "openvino>=2023.1.0" onnx

In [ ]:
import time
import warnings
from pathlib import Path

import cv2
import numpy as np
import openvino as ov
import torch
from torchvision.models.segmentation import lraspp_mobilenet_v3_large, LRASPP_MobileNet_V3_Large_Weights

# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)

from notebook_utils import segmentation_map_to_image, viz_result_image, SegmentationMap, Label, download_file

# Define the path for the ONNX model
onnx_path = Path('generator.onnx')

# Load the trained generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh(),
        )

    def forward(self, input):
        return self.main(input)

generator = Generator().to(device)
generator.load_state_dict(torch.load('generator_gpu.pth'))
generator.eval()  # Set the model to evaluation mode

# Suppress warnings and export the model to ONNX format if it doesn't already exist
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    if not onnx_path.exists():
        # Adjust the input dimensions to match your model
        dummy_input = torch.randn(1, 100).to(device)  # Adjusted input dimensions
        torch.onnx.export(
            generator,
            dummy_input,
            onnx_path,
        )
        print(f"ONNX model exported to {onnx_path}.")
    else:
        print(f"ONNX model {onnx_path} already exists.")


In [ ]:
import warnings
from pathlib import Path
import torch
import torch.onnx

# Define the model and input dimensions
IMAGE_WIDTH = 780
IMAGE_HEIGHT = 520
DIRECTORY_NAME = "model"
BASE_MODEL_NAME = DIRECTORY_NAME + "/generator"
weights_path = Path(BASE_MODEL_NAME + ".pt")

# Paths where ONNX and OpenVINO IR models will be stored.
onnx_path = weights_path.with_suffix('.onnx')
if not onnx_path.parent.exists():
    onnx_path.parent.mkdir()
ir_path = onnx_path.with_suffix(".xml")

# Load the trained generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh(),
        )

    def forward(self, input):
        return self.main(input)
generator = Generator().to(device)
generator.load_state_dict(torch.load('generator_gpu.pth'))
generator.eval()  # Set the model to evaluation mode

# Suppress warnings and export the model to ONNX format if it doesn't already exist
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    if not onnx_path.exists():
        # Adjust the input dimensions to match your model
        dummy_input = torch.randn(1, 100).to(device)  # Adjusted input dimensions
        torch.onnx.export(
            generator,
            dummy_input,
            onnx_path,
        )
        print(f"ONNX model exported to {onnx_path}.")
    else:
        print(f"ONNX model {onnx_path} already exists.")


In [ ]:
if not ir_path.exists():
    print("Exporting ONNX model to IR... This may take a few minutes.")
    ov_model = ov.convert_model(onnx_path)
    ov.save_model(ov_model, ir_path)
else:
    print(f"IR model {ir_path} already exists.")